# COMMENTS

In [11]:
# ============================================
# Install & Import Dependencies
# ============================================
# !pip install scipy pandas tensorflow

import pandas as pd
import numpy as np
from scipy.io import loadmat
from datetime import datetime, timedelta


# ============================================
# Helper Function: MATLAB datenum → datetime
# ============================================
def matlab2datetime(matlab_datenum):
    return datetime.fromordinal(int(matlab_datenum)) \
           + timedelta(days=matlab_datenum % 1) \
           - timedelta(days=366)

# ============================================
# Load .mat Dataset
# ============================================
data = loadmat('NEUSTG_19502020_12stations.mat')

lat = data['lattg'].flatten()
lon = data['lontg'].flatten()
sea_level = data['sltg']
station_names = [s[0] for s in data['sname'].flatten()]
time = data['t'].flatten()
time_dt = np.array([matlab2datetime(t) for t in time])

# ============================================
# Select Target Stations (Training & Testing)
# ============================================
TRAINING_STATIONS = ['Annapolis','Atlantic_City','Charleston','Washington','Wilmington', 'Eastport', 'Portland', 'Sewells_Point', 'Sandy_Hook']
TESTING_STATIONS = ['Lewes', 'Fernandina_Beach', 'The_Battery']

# Combine all stations for data loading
ALL_STATIONS = TRAINING_STATIONS + TESTING_STATIONS

selected_idx = [station_names.index(st) for st in ALL_STATIONS]
selected_names = [station_names[i] for i in selected_idx]
selected_lat = lat[selected_idx]
selected_lon = lon[selected_idx]
selected_sea_level = sea_level[:, selected_idx]  # time × selected_stations

print(f"Training stations: {TRAINING_STATIONS}")
print(f"Testing stations: {TESTING_STATIONS}")

# ============================================
# Build Preview DataFrame
# ============================================
df_preview = pd.DataFrame({
    'time': np.tile(time_dt[:5], len(selected_names)),
    'station_name': np.repeat(selected_names, 5),
    'latitude': np.repeat(selected_lat, 5),
    'longitude': np.repeat(selected_lon, 5),
    'sea_level': selected_sea_level[:5, :].T.flatten()
})

# ============================================
# Print Data Head
# ============================================
print(f"Number of stations: {len(selected_names)}")
print(f"Sea level shape (time x stations): {selected_sea_level.shape}")
df_preview.head()

Training stations: ['Annapolis', 'Atlantic_City', 'Charleston', 'Washington', 'Wilmington', 'Eastport', 'Portland', 'Sewells_Point', 'Sandy_Hook']
Testing stations: ['Lewes', 'Fernandina_Beach', 'The_Battery']
Number of stations: 12
Sea level shape (time x stations): (622392, 12)


,time,station_name,latitude,longitude,sea_level
0,1950-01-01 00:00:00.000000,Annapolis,38.98328,-76.4816,1.341
1,1950-01-01 00:59:59.999997,Annapolis,38.98328,-76.4816,1.311
2,1950-01-01 02:00:00.000003,Annapolis,38.98328,-76.4816,1.280
3,1950-01-01 03:00:00.000000,Annapolis,38.98328,-76.4816,1.280
4,1950-01-01 03:59:59.999997,Annapolis,38.98328,-76.4816,1.341


In [12]:
# ============================================
# Convert Hourly → Daily per Station
# ============================================
# Convert time to pandas datetime
time_dt = pd.to_datetime(time_dt)

# Build hourly DataFrame for selected stations
df_hourly = pd.DataFrame({
    'time': np.tile(time_dt, len(selected_names)),
    'station_name': np.repeat(selected_names, len(time_dt)),
    'latitude': np.repeat(selected_lat, len(time_dt)),
    'longitude': np.repeat(selected_lon, len(time_dt)),
    'sea_level': selected_sea_level.flatten()
})

# ============================================
# Compute Flood Threshold per Station
# ============================================
threshold_df = df_hourly.groupby('station_name')['sea_level'].agg(['mean','std']).reset_index()
threshold_df['flood_threshold'] = threshold_df['mean'] + 1.5 * threshold_df['std']

df_hourly = df_hourly.merge(threshold_df[['station_name','flood_threshold']], on='station_name', how='left')

# ============================================
# Daily Aggregation + Flood Flag
# ============================================
df_daily = df_hourly.groupby(['station_name', pd.Grouper(key='time', freq='D')]).agg({
    'sea_level': 'mean',
    'latitude': 'first',
    'longitude': 'first',
    'flood_threshold': 'first'
}).reset_index()

# Flood flag: 1 if any hourly value exceeded threshold that day
hourly_max = df_hourly.groupby(['station_name', pd.Grouper(key='time', freq='D')])['sea_level'].max().reset_index()
df_daily = df_daily.merge(hourly_max, on=['station_name','time'], suffixes=('','_max'))
df_daily['flood'] = (df_daily['sea_level_max'] > df_daily['flood_threshold']).astype(int)

# ============================================
# Feature Engineering (3d & 7d means)
# ============================================
df_daily['sea_level_3d_mean'] = df_daily.groupby('station_name')['sea_level'].transform(
    lambda x: x.rolling(3, min_periods=1).mean())
df_daily['sea_level_7d_mean'] = df_daily.groupby('station_name')['sea_level'].transform(
    lambda x: x.rolling(7, min_periods=1).mean())

# Preview
df_daily.head()

,station_name,time,sea_level,latitude,longitude,flood_threshold,sea_level_max,flood,sea_level_3d_mean,sea_level_7d_mean
0,Annapolis,1950-01-01,2.299667,38.98328,-76.4816,3.945163,6.288,1,2.299667,2.299667
1,Annapolis,1950-01-02,1.941625,38.98328,-76.4816,3.945163,6.105,1,2.120646,2.120646
2,Annapolis,1950-01-03,1.562000,38.98328,-76.4816,3.945163,4.612,1,1.934431,1.934431
3,Annapolis,1950-01-04,1.518958,38.98328,-76.4816,3.945163,3.200,0,1.674194,1.830563
4,Annapolis,1950-01-05,1.922667,38.98328,-76.4816,3.945163,3.414,0,1.667875,1.848983


In [ ]:
# ============================================
# Build 7-day → 14-day Training Windows
# ============================================
FEATURES = ['sea_level', 'sea_level_3d_mean', 'sea_level_7d_mean']
HIST_DAYS = 7
FUTURE_DAYS = 14

X_train, y_train = [], []

for stn, grp in df_daily.groupby('station_name'):
    grp = grp.sort_values('time').reset_index(drop=True)
    for i in range(len(grp) - HIST_DAYS - FUTURE_DAYS):
        hist = grp.loc[i:i+HIST_DAYS-1, FEATURES].values.flatten()
        future = grp.loc[i+HIST_DAYS:i+HIST_DAYS+FUTURE_DAYS-1, 'flood'].values
        X_train.append(hist)
        y_train.append(future)

X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (233208, 21)
y_train shape: (233208, 14)
Training data from 9 training stations


In [14]:
# ============================================
# Load Prediction Windows from Seed File
# ============================================
# Read prediction windows from Seed_Historical_Time_Intervals.txt
pred_windows_df = pd.read_csv('Seed_Historical_Time_Intervals.txt', sep='\t')
pred_windows_df['start_date'] = pd.to_datetime(pred_windows_df['start_date'])
pred_windows_df['end_date'] = pd.to_datetime(pred_windows_df['end_date'])

print(f"Loaded {len(pred_windows_df)} prediction windows")
print("\nPrediction windows:")
print(pred_windows_df)

# ============================================
# Build X_test for All Windows
# ============================================
FEATURES = ['sea_level', 'sea_level_3d_mean', 'sea_level_7d_mean']
X_test_all = []
test_windows_info = []

for idx, row in pred_windows_df.iterrows():
    hist_start = row['start_date']
    hist_end = row['end_date']
    
    # Forecast period (14 days after historical window)
    test_start = hist_end + pd.Timedelta(days=1)
    test_end = test_start + pd.Timedelta(days=13)
    
    # Build X_test for this window (TESTING_STATIONS only)
    X_test_window = []
    for stn, grp in df_daily.groupby('station_name'):
        if stn in TESTING_STATIONS:
            mask = (grp['time'] >= hist_start) & (grp['time'] <= hist_end)
            hist_block = grp.loc[mask, FEATURES].values.flatten()
            if len(hist_block) == 7 * len(FEATURES):   # ensure full 7-day block
                X_test_window.append(hist_block)
    
    if len(X_test_window) > 0:
        X_test_all.append(np.array(X_test_window))
        test_windows_info.append({
            'window_idx': idx,
            'hist_start': hist_start,
            'hist_end': hist_end,
            'test_start': test_start,
            'test_end': test_end
        })

X_test = np.array(X_test_all)  # Shape: (num_windows, num_stations, 21)
print(f"\nX_test shape: {X_test.shape}  (windows × stations × {7*len(FEATURES)} features)")
print(f"Number of valid windows: {len(test_windows_info)}")

Loaded 15 prediction windows

Prediction windows:
   start_date   end_date
0  1962-03-06 1962-03-12
1  2013-07-21 2013-07-27
2  2011-05-13 2011-05-19
3  1995-12-21 1995-12-27
4  1995-09-05 1995-09-11
5  2009-12-31 2010-01-06
6  2020-09-16 2020-09-22
7  2013-10-07 2013-10-13
8  1958-04-03 1958-04-09
9  2011-05-13 2011-05-19
10 1988-04-08 1988-04-14
11 1996-12-04 1996-12-10
12 2003-04-14 2003-04-20
13 1979-01-25 1979-01-31
14 2015-03-18 2015-03-24

X_test shape: (15, 3, 21)  (windows × stations × 21 features)
Number of valid windows: 15


In [15]:
# ============================================
# Improved LSTM Model Implementation (PyTorch)
# ============================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef

# Reshape data for LSTM: (samples, timesteps, features)
FEATURES = ['sea_level', 'sea_level_3d_mean', 'sea_level_7d_mean']
X_train_lstm = X_train.reshape(-1, 7, len(FEATURES))

# X_test is now 3D: (num_windows, num_stations, 21 features)
# Reshape to (num_windows * num_stations, 7, 3) for LSTM
X_test_lstm = X_test.reshape(-1, 7, len(FEATURES))

# Normalize features (per feature across all timesteps and samples)
X_train_reshaped = X_train_lstm.reshape(-1, len(FEATURES))
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train_lstm.shape)

X_test_reshaped = X_test_lstm.reshape(-1, len(FEATURES))
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test_lstm.shape)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Improved PyTorch LSTM model with deeper architecture
class ImprovedLSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=64, num_layers=2, dropout=0.3, output_size=14):
        super(ImprovedLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Multi-layer LSTM with dropout
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        # Additional fully connected layers
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # LSTM forward pass
        lstm_out, _ = self.lstm(x)
        # Take the last output from LSTM
        last_out = lstm_out[:, -1, :]
        
        # Fully connected layers with batch norm and dropout
        out = self.fc1(last_out)
        out = self.bn(out)
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return self.sigmoid(out)

# Initialize improved model
lstm_model = ImprovedLSTMModel(
    input_size=len(FEATURES), 
    hidden_size=64, 
    num_layers=2,
    dropout=0.3,
    output_size=14
)

criterion = nn.BCELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Validation split
val_size = int(len(X_train_tensor) * 0.1)
train_size = len(X_train_tensor) - val_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

# Training with early stopping
num_epochs = 20
best_val_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    # Training phase
    lstm_model.train()
    train_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = lstm_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()
    
    # Validation phase
    lstm_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs = lstm_model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    
    # Learning rate scheduling
    scheduler.step(avg_val_loss)
    
    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Save best model
        torch.save(lstm_model.state_dict(), 'best_lstm_model.pth')
    else:
        patience_counter += 1
    
    if (epoch + 1) % 2 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
    
    if patience_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        # Load best model
        lstm_model.load_state_dict(torch.load('best_lstm_model.pth'))
        break

# Load best model for prediction
try:
    lstm_model.load_state_dict(torch.load('best_lstm_model.pth'))
except:
    pass

# Predict for all windows
lstm_model.eval()
with torch.no_grad():
    y_pred_lstm = lstm_model(X_test_tensor).numpy()
y_pred_lstm_bin = (y_pred_lstm > 0.5).astype(int)

# Reshape predictions back to (num_windows, num_stations, 14)
num_stations = len(TESTING_STATIONS)
num_windows = len(test_windows_info)
y_pred_lstm_bin = y_pred_lstm_bin.reshape(num_windows, num_stations, 14)

# Collect ground truth for all windows (TESTING_STATIONS only)
y_true_all = []
for window_info in test_windows_info:
    test_start = window_info['test_start']
    test_end = window_info['test_end']
    y_true_window = []
    for stn, grp in df_daily.groupby('station_name'):
        if stn in TESTING_STATIONS:
            mask = (grp['time'] >= test_start) & (grp['time'] <= test_end)
            vals = grp.loc[mask, 'flood'].values
            if len(vals) == 14:
                y_true_window.append(vals)
            else:
                # Pad with zeros if missing days
                y_true_window.append(np.zeros(14, dtype=int))
    y_true_all.append(np.array(y_true_window))

y_true_all = np.array(y_true_all)  # Shape: (num_windows, num_stations, 14)

# Aggregate results across all windows
y_true_flat = y_true_all.flatten()
y_pred_flat = y_pred_lstm_bin.flatten()

tn, fp, fn, tp = confusion_matrix(y_true_flat, y_pred_flat).ravel()
acc = accuracy_score(y_true_flat, y_pred_flat)
f1 = f1_score(y_true_flat, y_pred_flat)
mcc = matthews_corrcoef(y_true_flat, y_pred_flat)

print("\n=== Improved LSTM Results (PyTorch) - All Windows ===")
print(f"Total predictions: {len(y_true_flat)} (across {num_windows} windows × {num_stations} testing stations × 14 days)")
print(f"Testing stations: {TESTING_STATIONS}")
print(f"TP: {tp} | FP: {fp} | TN: {tn} | FN: {fn}")
print(f"Accuracy: {acc:.3f} | F1: {f1:.3f} | MCC: {mcc:.3f}")

# Per-window metrics
print("\n=== Per-Window Metrics ===")
for i, window_info in enumerate(test_windows_info):
    y_true_win = y_true_all[i].flatten()
    y_pred_win = y_pred_lstm_bin[i].flatten()
    acc_win = accuracy_score(y_true_win, y_pred_win)
    f1_win = f1_score(y_true_win, y_pred_win)
    print(f"Window {i+1} ({window_info['hist_start'].date()} - {window_info['hist_end'].date()}): "
          f"Acc={acc_win:.3f}, F1={f1_win:.3f}")


Epoch [1/20], Train Loss: 0.1583, Val Loss: 0.0991
Epoch [2/20], Train Loss: 0.1142, Val Loss: 0.0952
Epoch [4/20], Train Loss: 0.1078, Val Loss: 0.0905
Epoch [6/20], Train Loss: 0.1053, Val Loss: 0.0904
Epoch [8/20], Train Loss: 0.1041, Val Loss: 0.0882
Epoch [10/20], Train Loss: 0.1032, Val Loss: 0.0868
Epoch [12/20], Train Loss: 0.1025, Val Loss: 0.0865
Epoch [14/20], Train Loss: 0.1017, Val Loss: 0.0871
Epoch [16/20], Train Loss: 0.1012, Val Loss: 0.0884
Epoch [18/20], Train Loss: 0.1010, Val Loss: 0.0851
Epoch [20/20], Train Loss: 0.1006, Val Loss: 0.0865

=== Improved LSTM Results (PyTorch) - All Windows ===
Total predictions: 630 (across 15 windows × 3 testing stations × 14 days)
Testing stations: ['Lewes', 'Fernandina_Beach', 'The_Battery']
TP: 404 | FP: 8 | TN: 216 | FN: 2
Accuracy: 0.984 | F1: 0.988 | MCC: 0.965

=== Per-Window Metrics ===
Window 1 (1962-03-06 - 1962-03-12): Acc=1.000, F1=1.000
Window 2 (2013-07-21 - 2013-07-27): Acc=0.976, F1=0.982
Window 3 (2011-05-13 - 201